In [30]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import numpy as np

In [4]:
df =  pd.read_csv('CombinesDS_2.txt', delimiter='|')

In [5]:
df.head()

,Sentences,Present,Code Word,Target Word
0,"all up my nose already haha, was a very very f...",yes,crystal,methamphetamine
1,nah he extended his 'christmas' sale because h...,yes,crystal,methamphetamine
2,boy you really saved us all with one. almost w...,no,NaN,NaN
3,ask yourself if you would eat another apple,no,NaN,NaN
4,the consistency was very similar to baby powde...,yes,baby powder,cocaine


In [8]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''

    return TAG_RE.sub('', text)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [103]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''

    sentence = sen.lower()

    # Remove html tags
    sentence = remove_tags(sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    #pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    #sentence = pattern.sub('', sentence)

    return sentence

In [74]:
def append_label(word):
  if(word):
    word = "__label__" + word
    word = word.replace(' ','-')
  return word

In [104]:
df['Preprocessed_sentence']= df['Sentences'].apply(preprocess_text)

In [75]:
df['Present_labelled']= df['Present'].apply(append_label)

In [76]:
df['codeword_labelled']= df['Code Word'].apply(lambda x: append_label(x) if(np.all(pd.notnull(x))) else x)


In [113]:
df.head(10)

,Sentences,Present,Code Word,Target Word,Preprocessed_sentence,Present_labelled,codeword_labelled
0,"all up my nose already haha, was a very very f...",yes,crystal,methamphetamine,all up my nose already haha was very very fine...,__label__yes,__label__crystal
1,nah he extended his 'christmas' sale because h...,yes,crystal,methamphetamine,nah he extended his christmas sale because he ...,__label__yes,__label__crystal
2,boy you really saved us all with one. almost w...,no,NaN,NaN,boy you really saved us all with one almost we...,__label__no,NaN
3,ask yourself if you would eat another apple,no,NaN,NaN,ask yourself if you would eat another apple,__label__no,NaN
4,the consistency was very similar to baby powde...,yes,baby powder,cocaine,the consistency was very similar to baby powde...,__label__yes,__label__baby-powder
5,"my original review/prices were based on the ""f...",yes,fish scale,cocaine,my original review prices were based on the fi...,__label__yes,__label__fish-scale
6,she thought it was normal stomach issues/maybe...,no,NaN,NaN,she thought it was normal stomach issues maybe...,__label__no,NaN
7,i probably should have taken this advice when ...,yes,acid,lsd,i probably should have taken this advice when ...,__label__yes,__label__acid
8,enjoy! amnesia haze is my strain of choice! we...,yes,haze,lsd,enjoy amnesia haze is my strain of choice we d...,__label__yes,__label__haze
9,i think their point was that once you stop str...,no,NaN,NaN,i think their point was that once you stop str...,__label__no,NaN


In [179]:
df_train = df.iloc[103:,:]
df_test = df.iloc[:103,:]

In [180]:
df_train.drop(columns=['Sentences','Present','Code Word','Target Word'], inplace=True)
df_test.drop(columns=['Sentences','Present','Code Word','Target Word'], inplace=True)

<ipython-input-180-7586bc30d984>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=['Sentences','Present','Code Word','Target Word'], inplace=True)
<ipython-input-180-7586bc30d984>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(columns=['Sentences','Present','Code Word','Target Word'], inplace=True)


In [116]:
df_train = df_train[['Present_labelled','codeword_labelled','Preprocessed_sentence']]
df_test = df_test [['Present_labelled','codeword_labelled','Preprocessed_sentence']]

In [117]:
import csv
df_train.to_csv('Codeword_Train.txt', sep=' ',quoting=csv.QUOTE_NONE,escapechar = ' ', header=False, index=False)
df_test.to_csv('Codeword_Test.txt', sep=' ',quoting=csv.QUOTE_NONE,escapechar = ' ', header=False, index=False)

In [61]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199674 sha256=c107963fe08b1cb190653985f5166257cac0c5a696c4e13cd9b4c403f869d3b0
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [62]:
import fasttext

In [168]:
model = fasttext.train_supervised(input="Codeword_Train.txt", lr=0.7, epoch=25, wordNgrams=3,dim=50,bucket=200000,loss='ova')


In [171]:
model.predict("agreed  for  me  they  re  total  waste  of  weed  since  can  really  only  inhale  few  hits  or  else  ll  be  puking  my  guts  out",k=3)

(('__label__yes', '__label__no', '__label__weed'),
 array([0.65136486, 0.34159252, 0.10670059]))

In [181]:
model.test("Codeword_Test.txt")

(103, 0.5825242718446602, 0.4316546762589928)